In [1]:
import pandas as pd
from decimal import Decimal

from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo 5-2

In this demo we investigate the **Fast Router performance** against implemented code

Initialize a fast simulator

In [2]:
Sim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, raiseonerror=True)
FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

Here we restrain the orders to just the top ones such that we can test the exact algo alongside

In [3]:
test_orders = pd.read_csv('orders.csv', dtype=str)
test_orders = test_orders[test_orders.id.isin(['0','1','2','3','251','250','6','248','247','246','10'])].copy()
test_orders

,id,liquidity,lowest_rate,highest_rate,current_rate
0,0,254814732,256,257,257
1,1,253827078,255,256,256
2,2,252839424,254,255,255
3,3,251851770,253,254,254
6,6,248888808,250,251,251
10,10,244938192,246,247,247
246,246,245925846,247,248,248
247,247,246913500,248,249,249
248,248,247901154,249,250,250
250,250,249876462,251,252,252


To translate this order input (effectively p_a and p_b) into a meaningful order in the simulator we need to transform them
The primary reason for this is that the current_rate (akin to p_marg) starts at the highest_rate (that has been denoted p_a) but the simulator takes the p_marg as being the p_start value in this pair

In [4]:
for i in test_orders.index:
    Sim.add_strategy('ETH', Decimal(test_orders.liquidity[i]), 1/Decimal(test_orders.highest_rate[i]), 1/Decimal(test_orders.lowest_rate[i]),  0, 0, 0)
    FastSim.add_strategy('ETH', Decimal(test_orders.liquidity[i]), 1/Decimal(test_orders.highest_rate[i]), 1/Decimal(test_orders.lowest_rate[i]),  0, 0, 0)

FastSim.state()['orders']

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,254814732.0,254814732.0,ETH,False,0.003891,0.003906,0.003891,USDC per ETH,1
1,1,ETHUSDC,USDC,0.0,0.0,USDC,False,0.000000,0.000000,None,USDC per ETH,0
2,2,ETHUSDC,ETH,253827078.0,253827078.0,ETH,False,0.003906,0.003922,0.003906,USDC per ETH,3
3,3,ETHUSDC,USDC,0.0,0.0,USDC,False,0.000000,0.000000,None,USDC per ETH,2
4,4,ETHUSDC,ETH,252839424.0,252839424.0,ETH,False,0.003922,0.003937,0.003922,USDC per ETH,5
5,5,ETHUSDC,USDC,0.0,0.0,USDC,False,0.000000,0.000000,None,USDC per ETH,4
6,6,ETHUSDC,ETH,251851770.0,251851770.0,ETH,False,0.003937,0.003953,0.003937,USDC per ETH,7
7,7,ETHUSDC,USDC,0.0,0.0,USDC,False,0.000000,0.000000,None,USDC per ETH,6
8,8,ETHUSDC,ETH,248888808.0,248888808.0,ETH,False,0.003984,0.004000,0.003984,USDC per ETH,9
9,9,ETHUSDC,USDC,0.0,0.0,USDC,False,0.000000,0.000000,None,USDC per ETH,8


We can see that when we input the order with flipped rates we get p_marg == p_start.

Unfortunately this means that the pricing is flipped relative to the order input

In [5]:
FastSim.state()['orders']

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,254814732.0,254814732.0,ETH,False,0.003891,0.003906,0.003891,USDC per ETH,1
1,1,ETHUSDC,USDC,0.0,0.0,USDC,False,0.000000,0.000000,None,USDC per ETH,0
2,2,ETHUSDC,ETH,253827078.0,253827078.0,ETH,False,0.003906,0.003922,0.003906,USDC per ETH,3
3,3,ETHUSDC,USDC,0.0,0.0,USDC,False,0.000000,0.000000,None,USDC per ETH,2
4,4,ETHUSDC,ETH,252839424.0,252839424.0,ETH,False,0.003922,0.003937,0.003922,USDC per ETH,5
5,5,ETHUSDC,USDC,0.0,0.0,USDC,False,0.000000,0.000000,None,USDC per ETH,4
6,6,ETHUSDC,ETH,251851770.0,251851770.0,ETH,False,0.003937,0.003953,0.003937,USDC per ETH,7
7,7,ETHUSDC,USDC,0.0,0.0,USDC,False,0.000000,0.000000,None,USDC per ETH,6
8,8,ETHUSDC,ETH,248888808.0,248888808.0,ETH,False,0.003984,0.004000,0.003984,USDC per ETH,9
9,9,ETHUSDC,USDC,0.0,0.0,USDC,False,0.000000,0.000000,None,USDC per ETH,8


## Route by Source

#### AMM buys USDC

Lets do some simple checks against the exact algo and then verify against the fast router results

1. Trade an amount against exact

In [6]:
Sim.amm_buys('USDC',10000000, execute=False)['trades']  # route_trade_by_source

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,False,None,2.548147e+08,ETH,9.934316e+05,USDC,ETHUSDC,0,1,0.003969,USDC per ETH
0,0.1,0,1,route #2,False,False,None,2.538271e+08,ETH,9.934543e+05,USDC,ETHUSDC,2,1,0.003969,USDC per ETH
0,0.2,0,2,route #4,False,False,None,2.528394e+08,ETH,9.934771e+05,USDC,ETHUSDC,4,1,0.003969,USDC per ETH
0,0.3,0,3,route #6,False,False,None,2.518518e+08,ETH,9.935000e+05,USDC,ETHUSDC,6,1,0.003969,USDC per ETH
0,0.4,0,4,route #8,False,False,None,2.488888e+08,ETH,9.935701e+05,USDC,ETHUSDC,8,1,0.003969,USDC per ETH
0,0.5,0,5,route #18,False,False,None,2.498765e+08,ETH,9.935465e+05,USDC,ETHUSDC,18,1,0.003969,USDC per ETH
0,0.6,0,6,route #20,False,False,None,2.508641e+08,ETH,9.935232e+05,USDC,ETHUSDC,20,1,0.003969,USDC per ETH
0,0.7,0,7,route #14,False,False,None,2.469135e+08,ETH,9.936177e+05,USDC,ETHUSDC,14,1,0.003969,USDC per ETH
0,0.8,0,8,route #16,False,False,None,2.479012e+08,ETH,9.935938e+05,USDC,ETHUSDC,16,1,0.003969,USDC per ETH
0,0.9,0,9,route #12,False,False,None,2.459258e+08,ETH,9.936418e+05,USDC,ETHUSDC,12,1,0.003969,USDC per ETH


We can observe the price is 0.003969 USDC per ETH (ignoring this is non-realistic).

For trade_by_source, when we increase the trade amount we push further down the curve which means that effective price should get worse.

In [7]:
Sim.amm_buys('USDC',10500000, execute=False)['trades']  # route_trade_by_source

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #0,False,False,None,2.548147e+08,ETH,9.934316e+05,USDC,ETHUSDC,0,1,0.003973,USDC per ETH
0,1.1,1,1,route #2,False,False,None,2.538271e+08,ETH,9.934543e+05,USDC,ETHUSDC,2,1,0.003973,USDC per ETH
0,1.2,1,2,route #4,False,False,None,2.528394e+08,ETH,9.934771e+05,USDC,ETHUSDC,4,1,0.003973,USDC per ETH
0,1.3,1,3,route #6,False,False,None,2.518518e+08,ETH,9.935000e+05,USDC,ETHUSDC,6,1,0.003973,USDC per ETH
0,1.4,1,4,route #8,False,False,None,2.488888e+08,ETH,9.935701e+05,USDC,ETHUSDC,8,1,0.003973,USDC per ETH
0,1.5,1,5,route #18,False,False,None,2.498765e+08,ETH,9.935465e+05,USDC,ETHUSDC,18,1,0.003973,USDC per ETH
0,1.6,1,6,route #20,False,False,None,2.508641e+08,ETH,9.935232e+05,USDC,ETHUSDC,20,1,0.003973,USDC per ETH
0,1.7,1,7,route #14,False,False,None,2.469135e+08,ETH,9.936177e+05,USDC,ETHUSDC,14,1,0.003973,USDC per ETH
0,1.8,1,8,route #16,False,False,None,2.479012e+08,ETH,9.935938e+05,USDC,ETHUSDC,16,1,0.003973,USDC per ETH
0,1.9,1,9,route #12,False,False,None,2.459258e+08,ETH,9.936418e+05,USDC,ETHUSDC,12,1,0.003973,USDC per ETH


And indeed for the exact algo, the price gets larger (0.003973 vs 0.003969), i.e. for every y unit returned it cost more x units to get it

We can then do a similar thing for the fast router

In [8]:
FastSim.amm_buys('USDC',10000000, execute=False)['trades']  # route_trade_by_source

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,False,None,2.548147e+08,ETH,9.934326e+05,USDC,ETHUSDC,0,1,0.003969,USDC per ETH
0,0.1,0,1,route #2,False,False,None,2.538271e+08,ETH,9.934553e+05,USDC,ETHUSDC,2,1,0.003969,USDC per ETH
0,0.2,0,2,route #4,False,False,None,2.528394e+08,ETH,9.934781e+05,USDC,ETHUSDC,4,1,0.003969,USDC per ETH
0,0.3,0,3,route #6,False,False,None,2.518518e+08,ETH,9.935010e+05,USDC,ETHUSDC,6,1,0.003969,USDC per ETH
0,0.4,0,4,route #20,False,False,None,2.508641e+08,ETH,9.935242e+05,USDC,ETHUSDC,20,1,0.003969,USDC per ETH
0,0.5,0,5,route #18,False,False,None,2.498765e+08,ETH,9.935475e+05,USDC,ETHUSDC,18,1,0.003969,USDC per ETH
0,0.6,0,6,route #8,False,False,None,2.488888e+08,ETH,9.935711e+05,USDC,ETHUSDC,8,1,0.003969,USDC per ETH
0,0.7,0,7,route #16,False,False,None,2.479012e+08,ETH,9.935948e+05,USDC,ETHUSDC,16,1,0.003969,USDC per ETH
0,0.8,0,8,route #14,False,False,None,2.469135e+08,ETH,9.936187e+05,USDC,ETHUSDC,14,1,0.003969,USDC per ETH
0,0.9,0,9,route #12,False,False,None,2.459258e+08,ETH,9.936428e+05,USDC,ETHUSDC,12,1,0.003969,USDC per ETH


We see that the orders are filled (almost) the same and the corresponding price is as we saw before (0.003969 - despite being negative)

We can then trade the higher amount and again, expect to see the price increase

In [9]:
FastSim.amm_buys('USDC',10500000, execute=False)['trades']  # route_trade_by_source

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #0,False,False,None,2.548147e+08,ETH,9.934326e+05,USDC,ETHUSDC,0,1,0.003973,USDC per ETH
0,1.1,1,1,route #2,False,False,None,2.538271e+08,ETH,9.934553e+05,USDC,ETHUSDC,2,1,0.003973,USDC per ETH
0,1.2,1,2,route #4,False,False,None,2.528394e+08,ETH,9.934781e+05,USDC,ETHUSDC,4,1,0.003973,USDC per ETH
0,1.3,1,3,route #6,False,False,None,2.518518e+08,ETH,9.935010e+05,USDC,ETHUSDC,6,1,0.003973,USDC per ETH
0,1.4,1,4,route #20,False,False,None,2.508641e+08,ETH,9.935242e+05,USDC,ETHUSDC,20,1,0.003973,USDC per ETH
0,1.5,1,5,route #18,False,False,None,2.498765e+08,ETH,9.935475e+05,USDC,ETHUSDC,18,1,0.003973,USDC per ETH
0,1.6,1,6,route #8,False,False,None,2.488888e+08,ETH,9.935711e+05,USDC,ETHUSDC,8,1,0.003973,USDC per ETH
0,1.7,1,7,route #16,False,False,None,2.479012e+08,ETH,9.935948e+05,USDC,ETHUSDC,16,1,0.003973,USDC per ETH
0,1.8,1,8,route #14,False,False,None,2.469135e+08,ETH,9.936187e+05,USDC,ETHUSDC,14,1,0.003973,USDC per ETH
0,1.9,1,9,route #12,False,False,None,2.459258e+08,ETH,9.936428e+05,USDC,ETHUSDC,12,1,0.003973,USDC per ETH


Again, we see that the price has, as before, increased appropriately.

## Route by Target

#### AMM sells ETH

We can now test the route by target.

First we do the counter trade - selling the output amount of ETH from the initial USDC trade

In [10]:
Sim.amm_sells('ETH',2519666367, execute=False)['trades']  # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,2.0,2,0,route #0,False,False,None,2.548147e+08,ETH,9.934316e+05,USDC,ETHUSDC,0,1,0.003969,USDC per ETH
0,2.1,2,1,route #2,False,False,None,2.538271e+08,ETH,9.934543e+05,USDC,ETHUSDC,2,1,0.003969,USDC per ETH
0,2.2,2,2,route #4,False,False,None,2.528394e+08,ETH,9.934771e+05,USDC,ETHUSDC,4,1,0.003969,USDC per ETH
0,2.3,2,3,route #6,False,False,None,2.518518e+08,ETH,9.935000e+05,USDC,ETHUSDC,6,1,0.003969,USDC per ETH
0,2.4,2,4,route #18,False,False,None,2.498765e+08,ETH,9.935465e+05,USDC,ETHUSDC,18,1,0.003969,USDC per ETH
0,2.5,2,5,route #20,False,False,None,2.508641e+08,ETH,9.935232e+05,USDC,ETHUSDC,20,1,0.003969,USDC per ETH
0,2.6,2,6,route #8,False,False,None,2.488888e+08,ETH,9.935701e+05,USDC,ETHUSDC,8,1,0.003969,USDC per ETH
0,2.7,2,7,route #16,False,False,None,2.479012e+08,ETH,9.935938e+05,USDC,ETHUSDC,16,1,0.003969,USDC per ETH
0,2.8,2,8,route #14,False,False,None,2.469135e+08,ETH,9.936177e+05,USDC,ETHUSDC,14,1,0.003969,USDC per ETH
0,2.9,2,9,route #12,False,False,None,2.459258e+08,ETH,9.936418e+05,USDC,ETHUSDC,12,1,0.003969,USDC per ETH


And we see that the return amount is close to the 10000000 we saw before - but importantly so is the price

Now we expect that since we want want (as the trader) to arrive at a smaller amount of ETH in hand, then we take less off the curve and the price should get lower

In [11]:
Sim.amm_sells('ETH',1500000000, execute=False)['trades']  # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,3.0,3,0,route #0,False,False,None,2.548147e+08,ETH,9.934316e+05,USDC,ETHUSDC,0,1,0.003937,USDC per ETH
0,3.1,3,1,route #2,False,False,None,2.538271e+08,ETH,9.934543e+05,USDC,ETHUSDC,2,1,0.003937,USDC per ETH
0,3.2,3,2,route #4,False,False,None,2.528394e+08,ETH,9.934771e+05,USDC,ETHUSDC,4,1,0.003937,USDC per ETH
0,3.3,3,3,route #6,False,False,None,2.518518e+08,ETH,9.935000e+05,USDC,ETHUSDC,6,1,0.003937,USDC per ETH
0,3.4,3,4,route #20,False,False,None,2.508641e+08,ETH,9.935232e+05,USDC,ETHUSDC,20,1,0.003937,USDC per ETH
0,3.5,3,5,route #18,False,False,None,2.358029e+08,ETH,9.374828e+05,USDC,ETHUSDC,18,1,0.003937,USDC per ETH
0,3,3,A,AMM sells 1500000000ETH buys 5904869USDC,True,False,None,1.500000e+09,ETH,5.904869e+06,USDC,ETHUSDC,"[0, 2, 4, 6, 20, 18]",6,0.003937,USDC per ETH


Indeed we see that the price for a smaller amount is less (0.003937 compared to 0.003969)

Now we do the same for the Fast router. Trade in the previous output amount

In [12]:
FastSim.amm_sells('ETH',2519666367, execute=False)['trades']  # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,2.0,2,0,route #0,False,False,None,2.548147e+08,ETH,9.934326e+05,USDC,ETHUSDC,0,1,0.00396878145395729112433288190694171992601010...,USDC per ETH
0,2.1,2,1,route #2,False,False,None,2.538271e+08,ETH,9.934553e+05,USDC,ETHUSDC,2,1,0.00396878145395729112433288190694171992601010...,USDC per ETH
0,2.2,2,2,route #4,False,False,None,2.528394e+08,ETH,9.934781e+05,USDC,ETHUSDC,4,1,0.00396878145395729112433288190694171992601010...,USDC per ETH
0,2.3,2,3,route #6,False,False,None,2.518518e+08,ETH,9.935010e+05,USDC,ETHUSDC,6,1,0.00396878145395729112433288190694171992601010...,USDC per ETH
0,2.4,2,4,route #20,False,False,None,2.508641e+08,ETH,9.935242e+05,USDC,ETHUSDC,20,1,0.00396878145395729112433288190694171992601010...,USDC per ETH
0,2.5,2,5,route #18,False,False,None,2.498765e+08,ETH,9.935475e+05,USDC,ETHUSDC,18,1,0.00396878145395729112433288190694171992601010...,USDC per ETH
0,2.6,2,6,route #8,False,False,None,2.488888e+08,ETH,9.935711e+05,USDC,ETHUSDC,8,1,0.00396878145395729112433288190694171992601010...,USDC per ETH
0,2.7,2,7,route #16,False,False,None,2.479012e+08,ETH,9.935948e+05,USDC,ETHUSDC,16,1,0.00396878145395729112433288190694171992601010...,USDC per ETH
0,2.8,2,8,route #14,False,False,None,2.469135e+08,ETH,9.936187e+05,USDC,ETHUSDC,14,1,0.00396878145395729112433288190694171992601010...,USDC per ETH
0,2.9,2,9,route #12,False,False,None,2.459258e+08,ETH,9.936428e+05,USDC,ETHUSDC,12,1,0.00396878145395729112433288190694171992601010...,USDC per ETH


And importantly we see a similar price to the exact algo

The test now is to see the price go down accordingly as the request amount is lowered

In [13]:
FastSim.amm_sells('ETH',1500000000, execute=False)['trades']  # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,3.0,3,0,route #0,False,False,None,2.548147e+08,ETH,9.934326e+05,USDC,ETHUSDC,0,1,0.00393658332745240605588537878383418790166607...,USDC per ETH
0,3.1,3,1,route #2,False,False,None,2.538271e+08,ETH,9.934553e+05,USDC,ETHUSDC,2,1,0.00393658332745240605588537878383418790166607...,USDC per ETH
0,3.2,3,2,route #4,False,False,None,2.528394e+08,ETH,9.934781e+05,USDC,ETHUSDC,4,1,0.00393658332745240605588537878383418790166607...,USDC per ETH
0,3.3,3,3,route #6,False,False,None,2.518518e+08,ETH,9.935010e+05,USDC,ETHUSDC,6,1,0.00393658332745240605588537878383418790166607...,USDC per ETH
0,3.4,3,4,route #20,False,False,None,2.508641e+08,ETH,9.935242e+05,USDC,ETHUSDC,20,1,0.00393658332745240605588537878383418790166607...,USDC per ETH
0,3.5,3,5,route #18,False,False,None,2.358029e+08,ETH,9.374838e+05,USDC,ETHUSDC,18,1,0.00393658332745240605588537878383418790166607...,USDC per ETH
0,3,3,A,AMM sells 5904875ETH buys 1500000000USDC,True,False,None,5.904875e+06,ETH,1.500000e+09,USDC,ETHUSDC,"[0, 2, 4, 6, 20, 18]",6,0.00393658332745240589073005996640685124851869...,USDC per ETH


Here we get 0.003936 which is both less than the larger trade amount and roughly the same as the exact method.